In [32]:
import requests, json, urllib.parse

base_url = 'https://itunes.apple.com/search'

artist = 'Little Big'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

print(list(data[0].keys()))

import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)

# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))

# Уникальные названия артистов
print(set(map(lambda x: x['artistName'], data)))

# Фильтрация с целью оставить искомого артиста
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)
print(set(map(lambda x: x['artistName'], data)))

# Функция переводящая миллисекунды в читаемый формат времени
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

# Удаление переменной, действие опционально
del sorted_by_album

print(list(album_names.values()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']
['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']
{'Little Big & Ruki Vverkh', 'Daddy Yankee, Lil Jon, Noriega & Pitbull', 'Ariana Grande', 'Darktek & Little Big', 'Lionel Richie', 'Lil Wayne', 'Miranda Lambert', 'Lil Jon & The East Si

In [64]:
import statistics
print(f"Artist: {data[0]['artistName']}")
print(f"Songs number: {len(set(map(lambda x: x['trackId'], data)))}")
print(f"Popular genre: {max(set(map(lambda x: x['primaryGenreName'], data)), key=list(map(lambda x: x['primaryGenreName'], data)).count)}")
print(f"Album amount: {len(set(map(lambda x: x['collectionName'], data)))}")
print(f"Total time: {convert_millis(sum(list(map(lambda x: x['trackTimeMillis'], data))))}")
print(f"Average time: {convert_millis(int(sum(list(map(lambda x: x['trackTimeMillis'], data)))/len(list(map(lambda x: x['trackTimeMillis'], data)))))}")
print(f"Median time: {convert_millis(int(statistics.median(list(map(lambda x: x['trackTimeMillis'], data)))))}")
print(f"Longest: {max(data, key=lambda x: x['trackTimeMillis'])['trackName']}")
print(f"Time: {convert_millis(max(data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])}")
print(f"Shortest: {min(data, key=lambda x: x['trackTimeMillis'])['trackName']}")
print(f"Time: {convert_millis(min(data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])}")
print(f"Total price: {sum(list(map(lambda x: x['trackPrice'], data)))}")
print(f"Average price: {sum(list(map(lambda x: x['trackPrice'], data)))/len(list(map(lambda x: x['trackPrice'], data)))}")
print(f"Median price: {statistics.median(list(map(lambda x: x['trackPrice'], data)))}")
print(f"Most expensive: {max(data, key=lambda x: x['trackPrice'])['trackName']}")
print(f"Price: {max(data, key=lambda x: x['trackPrice'])['trackPrice']}")
print(f"Cheapest: {min(data, key=lambda x: x['trackPrice'])['trackName']}")
print(f"Price: {min(data, key=lambda x: x['trackPrice'])['trackPrice']}")
#######
print("---------------------")
#######
album_data = list(filter(lambda x: 'Antipositive, Pt. 1' in x['collectionName'], data))
print(f"Album name: {album_data[0]['collectionName']}")
print(f"Songs amount: {len([a['trackName'] for a in album_data])}")
print(f"Popular genre: {max([a['primaryGenreName'] for a in album_data], key=[a['primaryGenreName'] for a in album_data].count)}")
print(f"Release date: {album_data[0]['releaseDate']}")
print(f"Total time: {convert_millis(sum([a['trackTimeMillis'] for a in album_data]))}")
print(f"Average time: {convert_millis(int(sum([a['trackTimeMillis'] for a in album_data])/len([a['trackTimeMillis'] for a in album_data])))}")
print(f"Median time: {convert_millis(int(statistics.median(([a['trackTimeMillis'] for a in album_data]))))}")
print(f"Longest song: {max(album_data, key=lambda x: x['trackTimeMillis'])['trackName']}")
print(f"Time: {convert_millis(max(album_data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])}")
print(f"Shortest song: {min(album_data, key=lambda x: x['trackTimeMillis'])['trackName']}")
print(f"Time: {convert_millis(min(album_data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])}")
print(f"Total price: {sum([a['trackPrice'] for a in album_data])}")
print(f"Average price: {sum([a['trackPrice'] for a in album_data])/len([a['trackPrice'] for a in album_data])}")
print(f"Median price: {statistics.median([a['trackPrice'] for a in album_data])}")
print(f"Most expensive: {max(album_data, key=lambda x: x['trackPrice'])['trackName']}")
print(f"Price: {(max(album_data, key=lambda x: x['trackPrice'])['trackPrice'])}")
print(f"Cheapest: {min(album_data, key=lambda x: x['trackPrice'])['trackName']}")
print(f"Price: {(min(album_data, key=lambda x: x['trackPrice'])['trackPrice'])}")

Artist: Little Big
Songs number: 187
Popular genre: Country
Album amount: 50
Total time: 10H59M14S
Average time: 0H3M31S
Median time: 0H3M28S
Longest: Skibidi (Doorly Remix)
Time: 0H5M22S
Shortest: Tacos (Lubim Remix)
Time: 0H1M37S
Total price: 230.56999999999968
Average price: 1.2329946524064155
Median price: 1.29
Most expensive: Skibidi
Price: 1.29
Cheapest: Uno (Eurovision 2020 / Russia)
Price: -1.0
---------------------
Album name: Antipositive, Pt. 1
Songs amount: 8
Popular genre: Dance
Release date: 2018-05-08 12:00:00+00:00
Total time: 0H24M12S
Average time: 0H3M1S
Median time: 0H3M1S
Longest song: Lolly Bomb
Time: 0H3M53S
Shortest song: Hate You
Time: 0H2M4S
Total price: 10.32
Average price: 1.29
Median price: 1.29
Most expensive: Faradenza
Price: 1.29
Cheapest: Faradenza
Price: 1.29
